In [1]:
import cv2
import numpy as np
import os

In [2]:
# Create output directory
os.makedirs("output", exist_ok=True)

# Load image (use a binary or shapes image)
img = cv2.imread("C:/Users/bushr/Desktop/MORPHOLOGICAL IMAGE.png", cv2.IMREAD_GRAYSCALE)

if img is None:
    raise FileNotFoundError("Image not found. Check image path.")

In [3]:
# Convert to binary
_, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

In [4]:
# Structuring element
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

In [5]:
# TASK 1: Erosion, Dilation, Opening, Closing
# ==================================================

erosion = cv2.erode(binary, kernel, iterations=1)
dilation = cv2.dilate(binary, kernel, iterations=1)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

cv2.imwrite("output/erosion.png", erosion)
cv2.imwrite("output/dilation.png", dilation)
cv2.imwrite("output/opening.png", opening)
cv2.imwrite("output/closing.png", closing)

True

In [6]:
# ==================================================
# TASK 2: Boundary Extraction
# ==================================================

boundary = binary - erosion
cv2.imwrite("output/boundaries.png", boundary)

True

In [7]:
# ==================================================
# TASK 3: Hole Filling using Morphological Reconstruction
# ==================================================

inv = cv2.bitwise_not(binary)
marker = np.zeros_like(binary)
marker[0, :] = inv[0, :]
marker[-1, :] = inv[-1, :]
marker[:, 0] = inv[:, 0]
marker[:, -1] = inv[:, -1]

filled = inv.copy()
while True:
    prev = filled.copy()
    filled = cv2.dilate(filled, kernel)
    filled = cv2.min(filled, inv)
    if np.array_equal(filled, prev):
        break

holes_filled = cv2.bitwise_not(filled)
cv2.imwrite("output/holes_filled.png", holes_filled)

True

In [8]:
# ==================================================
# TASK 4: Noise Removal using Morphology
# ==================================================

noise_removed = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
cv2.imwrite("output/noise_removed.png", noise_removed)

True

In [10]:
# TASK 5: Shape Detection
# ==================================================

contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
color_img = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

for cnt in contours:
    area = cv2.contourArea(cnt)
    if area < 500:
        continue

    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.04 * peri, True)

    if len(approx) == 3:
        shape = "Triangle"
    elif len(approx) == 4:
        shape = "Rectangle"
    else:
        shape = "Circle"

    cv2.drawContours(color_img, [cnt], -1, (0, 255, 0), 2)
    x, y = cnt[0][0]
    cv2.putText(color_img, shape, (x, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

cv2.imwrite("output/shape_detection.png", color_img)

print("✅ LAB 11 COMPLETED SUCCESSFULLY!")


✅ LAB 11 COMPLETED SUCCESSFULLY!
